In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
import sys
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.eval import calc_f1
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [5]:
import tensorflow as tf
from absl import flags
FLAGS = flags.FLAGS
from transformers import AutoTokenizer
from src import config
from src.model import Model
from src.dataset import Dataset
from src.util import get_preds
import src.eval as ev
import melt as mt
import husky

In [3]:
FLAGS([''])
FLAGS.mts = True
config.init_()
config.init()
mt.init()
model = Model()

ic| config.py:54 in init_()
    FLAGS.backbone: '../input/allenai/longformer-base-4096'
ic| config.py:54 in init_()
    FLAGS.backbone: '../input/allenai/longformer-base-4096'
ic| config.py:63 in init()
    FLAGS.valid_files[:2]: ['../input/feedback-prize-2021/tfrecords/train/70.164.tfrec',
                            '../input/feedback-prize-2021/tfrecords/train/90.164.tfrec']
ic| init.py:200 in init()
    tf.__version__: '2.6.2'
    torch.__version__: '1.9.0'
ic| init.py:334 in init()
    FLAGS.model_dir: '../working/offline/0/base.20211227-083507'
    FLAGS.log_dir: '../working/offline/0/base.20211227-083507'
2021-12-27 08:35:08 0:00:00 batch size is shrink by 4 for each gpu to make total insts per step still 32
2021-12-27 08:35:08 0:00:00 fcntl.floc with lock_file /home/huigecheng/.melt.lock (If hang here means other programs calling melt.init have not finished yet)
2021-12-27 08:35:08 0:00:00 Tf dataset and Tf model train in Eager mode, keras True, distributed:False
2021-12-27 08:

In [6]:
fit = mt.fit
strategy = mt.distributed.get_strategy()
with strategy.scope():    
  model = Model()
  
  fit(model,  
      Dataset=Dataset,
      eval_fn=ev.evaluate
      ) 

All model checkpoint layers were used when initializing TFLongformerModel.

All the layers of TFLongformerModel were initialized from the model checkpoint at ../input/allenai/longformer-base-4096.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerModel for predictions without further training.
ic| model.py:30 in __init__()- num_classes: 15
2021-12-27 08:36:22 0:01:14 Round: 0 mode: None train_input:[None] valid_input:[None] train_dirs:[1] valid_dir: None do_valid: True do_test: True
ic| train.py:1082 in train()
    len(inputs): 76
    inputs[:2]: ['../input/feedback-prize-2021/tfrecords/train/78.164.tfrec',
                 '../input/feedback-prize-2021/tfrecords/train/14.164.tfrec']
2021-12-27 08:36:32 0:01:24 Model: "model_2"
2021-12-27 08:36:32 0:01:24 _________________________________________________________________
2021-12-27 08:36:32 0:01:24 Layer (type)                 Output Shape              Param #   
2021-12-27 0

get_num_records:   0%|          | 0/76 [00:00<?, ?it/s]

ic| train.py:1192 in train()
    len(valid_inputs): 19
    valid_inputs[:2]: ['../input/feedback-prize-2021/tfrecords/train/0.165.tfrec',
                       '../input/feedback-prize-2021/tfrecords/train/5.165.tfrec']
2021-12-27 08:36:33 0:01:25 num_train_examples: 12475 num_steps_per_epoch: 390


get_num_records:   0%|          | 0/19 [00:00<?, ?it/s]

2021-12-27 08:36:33 0:01:25 num_valid_examples: 3119 num_valid_steps_per_epoch: 98
2021-12-27 08:36:33 0:01:25 round: 0 loss_fn: <function Model.get_loss_fn.<locals>.loss_fn at 0x7f5efb696598>
2021-12-27 08:36:33 0:01:25 optimizer: <husky.optimization.AdamWeightDecay object at 0x7f5efb5185f8> lr: <husky.optimization.WarmUp object at 0x7f5efb5180f0> init_lr: 0.0001
2021-12-27 08:36:33 0:01:25 total_steps: 1170 warmup_steps: 117 end_lr: 1e-07
2021-12-27 08:36:34 0:01:25 before loading, total params: 148776506, l2:0.003339
2021-12-27 08:36:34 0:01:25 No model.h5 or other weights file found in ../working/offline/0/base.20211227-083507


Training:   0%|          | 0/3 [00:00<?, ?epochs/s]

[base.20211227-083507] Epochs:3:   0%|          | 0/1170 [00:00<?, ?it/s]

In [ ]:
model.build_model().save_weights(f'{FLAGS.model_dir}/model2.h5')
    
dataset_meta_root = '..'
os.system(f'cp {dataset_meta_root}/dataset-metadata.json {FLAGS.model_dir}')    